## 移除最后conv最后一层网络

把 resnet最后一层再拼接回去，重新开始训练 ，finetune


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 256
num_epochs = 300
extract_size = 1024
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut()
encoder = encoder.to(device)

# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
# data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_conv.h")
data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_remove_last_conv.h')

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [4]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm1layer_last.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [5]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(1, num_epochs+1):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm1layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_multify_rm1layer_last.pkl'))

Epoch [1/300], Step [120/127], Loss: 3.4076, Accuracy: 14.45%, Best_acc: 14.45%            
 Epoch 1, spent time:5.86s
Epoch [2/300], Step [120/127], Loss: 2.2835, Accuracy: 37.89%, Best_acc: 37.89%            
 Epoch 2, spent time:5.69s
Epoch [3/300], Step [120/127], Loss: 1.9231, Accuracy: 48.83%, Best_acc: 48.83%            
 Epoch 3, spent time:5.72s
Epoch [4/300], Step [120/127], Loss: 1.4758, Accuracy: 55.86%, Best_acc: 57.42%            
 Epoch 4, spent time:5.60s
Epoch [5/300], Step [120/127], Loss: 1.2302, Accuracy: 66.02%, Best_acc: 66.02%            
 Epoch 5, spent time:5.69s
Epoch [6/300], Step [120/127], Loss: 1.0666, Accuracy: 66.41%, Best_acc: 68.36%            
 Epoch 6, spent time:5.65s
Epoch [7/300], Step [120/127], Loss: 0.9230, Accuracy: 70.31%, Best_acc: 72.66%            
 Epoch 7, spent time:5.68s
Epoch [8/300], Step [120/127], Loss: 0.9444, Accuracy: 67.97%, Best_acc: 72.66%            
 Epoch 8, spent time:5.67s
Epoch [9/300], Step [120/127], Loss: 0.9138, Acc

Epoch [69/300], Step [120/127], Loss: 0.3593, Accuracy: 82.81%, Best_acc: 89.06%            
 Epoch 69, spent time:5.73s
Epoch [70/300], Step [120/127], Loss: 0.3789, Accuracy: 82.03%, Best_acc: 89.06%            
 Epoch 70, spent time:5.76s
Epoch [71/300], Step [120/127], Loss: 0.3344, Accuracy: 85.55%, Best_acc: 89.06%            
 Epoch 71, spent time:5.80s
Epoch [72/300], Step [120/127], Loss: 0.3436, Accuracy: 87.50%, Best_acc: 89.06%            
 Epoch 72, spent time:5.76s
Epoch [73/300], Step [120/127], Loss: 0.3829, Accuracy: 81.64%, Best_acc: 89.06%            
 Epoch 73, spent time:5.74s
Epoch [74/300], Step [120/127], Loss: 0.3378, Accuracy: 84.77%, Best_acc: 89.06%            
 Epoch 74, spent time:5.74s
Epoch [75/300], Step [120/127], Loss: 0.3512, Accuracy: 86.33%, Best_acc: 90.62%            
 Epoch 75, spent time:5.74s
Epoch [76/300], Step [120/127], Loss: 0.3913, Accuracy: 83.20%, Best_acc: 90.62%            
 Epoch 76, spent time:5.74s
Epoch [77/300], Step [120/127], 

Epoch [202/300], Step [120/127], Loss: 0.1823, Accuracy: 90.62%, Best_acc: 95.70%            
 Epoch 202, spent time:5.87s
Epoch [203/300], Step [120/127], Loss: 0.2316, Accuracy: 90.23%, Best_acc: 95.70%            
 Epoch 203, spent time:5.78s
Epoch [204/300], Step [120/127], Loss: 0.2302, Accuracy: 91.41%, Best_acc: 95.70%            
 Epoch 204, spent time:5.74s
Epoch [205/300], Step [120/127], Loss: 0.1607, Accuracy: 92.97%, Best_acc: 95.70%            
 Epoch 205, spent time:5.68s
Epoch [206/300], Step [120/127], Loss: 0.2206, Accuracy: 90.62%, Best_acc: 95.70%            
 Epoch 206, spent time:5.72s
Epoch [207/300], Step [120/127], Loss: 0.2092, Accuracy: 92.58%, Best_acc: 95.70%            
 Epoch 207, spent time:5.63s
Epoch [208/300], Step [120/127], Loss: 0.2134, Accuracy: 92.58%, Best_acc: 95.70%            
 Epoch 208, spent time:5.81s
Epoch [209/300], Step [120/127], Loss: 0.2412, Accuracy: 90.62%, Best_acc: 95.70%            
 Epoch 209, spent time:5.79s
Epoch [210/300],

### How good is the model?

In [6]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

In [21]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm1layer_100.pkl'))

In [22]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 77.56%.


## With 100 epoch the result is the best,yet.

In [23]:
!mv ./models/class_multify_rm1layer_100.pkl ./models/class_multify_rm1layer_good.pkl